In [ ]:
# pip install numpy pandas
# pip install trl
# pip install sentence_transformers
# pip install -U bitsandbytes
# pip install langchain langchain-community pypdf pdfplumber faiss-cpu
# pip install langchain-ollama
# pip install rank_bm25

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 이건 진짜로 그냥 필요없음
# pip install -U transformers
# pip install -U accelerate
# pip install kiwipiepy

In [1]:
import os
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer

from langchain_community.document_loaders import PyPDFLoader # 1.로드
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter # 2.청크
from langchain.embeddings import HuggingFaceEmbeddings # 3. 임베딩 모델
from langchain.vectorstores import FAISS # 3. 벡터 저장
from langchain.retrievers import BM25Retriever, EnsembleRetriever # 4. 검색 기법
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate

from transformers import pipeline

/home/minhai/.local/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# 1. 문서 로드 (pdf페이지로 나눠짐)
loader = PyPDFLoader("/home/minhai/2024-2-DSCD-Synnovation-3/한화 빅플러스재산종합보험 bigplus(2310)_03.pdf")
documents = loader.load()

In [3]:
# 2. 문서를 적절한 크기의 조각으로 청크 (split)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30) # 청크 중 중복되는 부분 크기
  #seprator 공백이면 공백 기준으로 청크를.. \n을 기본적으로 사용 (이거는 순차적으로 진행해줌)

chunks = text_splitter.split_documents(documents)
chunks = chunks[11:] #불필요한 청크 제외(목차 등)

# 각 청크에 id 부여
for i, chunk in enumerate(chunks):
    chunk.metadata['doc_id'] = i

print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in chunks)}")

생성된 텍스트 청크 수: 1719
각 청크의 길이: [222, 478, 487, 485, 124, 477, 432, 496, 494, 496, 251, 485, 498, 466, 500, 235, 482, 492, 293, 497, 499, 496, 235, 484, 475, 222, 493, 498, 496, 493, 481, 499, 496, 494, 282, 423, 452, 492, 496, 337, 457, 26, 494, 499, 498, 498, 499, 499, 499, 498, 498, 67, 26, 499, 498, 498, 499, 498, 499, 497, 493, 498, 498, 498, 468, 26, 498, 499, 499, 499, 499, 499, 498, 499, 499, 499, 498, 242, 26, 499, 497, 495, 499, 498, 497, 498, 499, 499, 498, 463, 26, 498, 498, 499, 499, 499, 498, 498, 499, 498, 492, 498, 256, 26, 498, 498, 499, 482, 499, 499, 496, 495, 498, 498, 272, 26, 499, 498, 498, 499, 499, 498, 499, 499, 498, 479, 131, 26, 498, 499, 499, 499, 499, 499, 498, 499, 62, 26, 496, 489, 93, 27, 498, 498, 293, 199, 27, 497, 498, 499, 228, 27, 499, 497, 493, 130, 27, 498, 499, 495, 149, 341, 52, 498, 330, 168, 440, 356, 278, 486, 332, 141, 27, 496, 226, 360, 27, 497, 497, 112, 27, 493, 80, 497, 253, 27, 496, 495, 58, 27, 498, 495, 186, 242, 494, 167, 28, 27, 496, 4

In [4]:
# qa 데이터셋을 위해 파일 저장 

# meatadata에서 정보 추출
os.path.split(chunks[0].metadata['source'])[1].split('_')[0]

# 메타데이터가 저장된 데이터 프레임 제작
final_docs = []
for i, doc in enumerate(chunks):
    new_doc = doc.copy()
    new_doc.metadata['doc_id'] = i # metadata에 doc id를 추가
    new_doc.metadata['source'] = str(os.path.split(new_doc.metadata['source'])[1].split('_')[0])
    # corp_name = str(os.path.split(new_doc.metadata['source'])[1].split('_')[0]) # metadata에서 정보를 추출하여 page_content에 추가
    # new_doc.page_content = f"{new_doc.page_content}\n\n(참고: 이 문서는 {corp_name}에 대한 정보를 담고 있습니다.)"
    final_docs.append(new_doc)

# 문서 저장
import json
with open("chunks.json", "wb") as f:
    for doc in chunks:
        f.write(json.dumps(dict(doc)).encode('utf-8'))
        f.write(b'\n')

# JSONL 파일 로드하기
from langchain_community.document_loaders import JSONLoader

def metadata_func(record: dict, metadata: dict) -> dict: #메타 데이터 추가
    metadata = record.get("metadata")
    return metadata

json_loader = JSONLoader(
    file_path="/home/minhai/2024-2-DSCD-Synnovation-3/chunks.json",
    jq_schema=".",
    content_key="page_content",
    json_lines=True,
    metadata_func=metadata_func,
)

json_docs = json_loader.load()
print("문서의 수:", len(json_docs))

# 데이터프레임으로 변환
test_data = []
for doc in json_docs:
    test_data.append({
        'context': str(doc.page_content),
        'source': str(doc.metadata.get('source', '')),
        'doc_id': str(doc.metadata.get('doc_id', ''))
    })

df_test = pd.DataFrame(test_data)
print(df_test.shape)
df_test.head()
df_test.to_csv("/home/minhai/2024-2-DSCD-Synnovation-3/손해보험_chunks.csv")

문서의 수: 1719
(1719, 3)
